<a href="https://colab.research.google.com/github/Mohamed-El10/Mohamed-El10/blob/main/AnalyseSetimentWithSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files


uploaded_train = files.upload()
uploaded_test = files.upload()


Saving Train1.xlsx to Train1.xlsx


Saving Test1.xlsx to Test1.xlsx


In [31]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt') # Used for sentence tokenizer
nltk.download('stopwords')
nltk.download('wordnet')
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# train Data
trainData = pd.read_excel("Train1.xlsx")

# test Data
testData = pd.read_excel("Test1.xlsx")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [32]:
trainData.head() # shuffle the df and pick first 5


,Label,Content
0,pos,zero effect gets its title from the main chara...
1,pos,"you've seen this moment before , recently : a ..."
2,pos,you've probably heard the one about the priest...
3,pos,you've heard all the hype . \nyou've seen all ...
4,pos,you've got to love disney . \nno matter what t...


In [33]:
#pretraitement traindata
processed_lines = []
for line in trainData['Content']:

    tokens = word_tokenize(line.lower())

    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]

    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]

    # filter out stop words
    #stop_words = set(stopwords.words('english'))
    #words = [w for w in words if not w in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in words]

    cleaned_line = ' '.join(lemmatized_tokens)

    # Append the cleaned line to the list
    processed_lines.append(cleaned_line)

trainData['Content'] =processed_lines

In [34]:
trainData.head()

,Label,Content
0,pos,zero effect get it title from the main charact...
1,pos,you ve seen this moment before recently a part...
2,pos,you ve probably heard the one about the priest...
3,pos,you ve heard all the hype you ve seen all thei...
4,pos,you ve got to love disney no matter what they ...


In [35]:
#pretraitement testdata
processed_line = []
for line in testData['Content']:

    tokens = word_tokenize(line.lower())

    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]

    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]

    # filter out stop words
    #stop_words = set(stopwords.words('english'))
    #words = [w for w in words if not w in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in words]

    cleaned_line = ' '.join(lemmatized_tokens)

    # Append the cleaned line to the list
    processed_line.append(cleaned_line)

testData['Content'] =processed_line

In [36]:
testData.head()

,Label,Content
0,pos,you ve got to think twice before you go see a ...
1,pos,youssef ha an amazing hotel we could nt have a...
2,neg,you know something christmas is not about pres...
3,neg,yet another brainless teen flick this one is a...
4,pos,wyatt earp ha a lot to tell and little to say ...


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(trainData['Content'])
test_vectors = vectorizer.transform(testData['Content'])

In [38]:
import time
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report ,confusion_matrix
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, trainData['Label'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("\n\nTraining time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(testData['Label'], prediction_linear)
confusion_mat = confusion_matrix(testData['Label'], prediction_linear)


print("\n\nConfusion Matrix is :")
print(confusion_mat)
print("\nClassification Report is :\n",report)
#print('positive: ', report['pos'])
#print('negative: ', report['neg'])




Training time: 12.819129s; Prediction time: 1.643059s


Confusion Matrix is :
[[174  12]
 [ 12 262]]

Classification Report is :
               precision    recall  f1-score   support

         neg       0.94      0.94      0.94       186
         pos       0.96      0.96      0.96       274

    accuracy                           0.95       460
   macro avg       0.95      0.95      0.95       460
weighted avg       0.95      0.95      0.95       460



In [ ]:

review = """Do not purchase this product. My cell phone blast when I switched the charger"""
review_vector = vectorizer.transform([review]) # vectorizing
print(classifier_linear.predict(review_vector))
review = """It's not even 5 days since i purchased this product.
I would say this a specially blended worst Phone in all formats.
ISSUE 1:
Have you ever heard of phone which gets drained even in standby mode during night?
Kindly please see the screenshot if you want to believe my statement.
My phone was in full charge at night 10:07 PM . I took this screenshot and went to sleep.
Then I woke up at morning and 6:35 AM and battery got drained by 56% in just standby condition.
If this is the case consider how many hours it will work, during day time.
It's not even 5 hours the battery is able to withstand.
ISSUE 2:
Apart from the battery, the next issue is the heating issue .I purchased a iron box recently from Bajaj in this sale.
But I realized this phone acts a very good Iron box than the Bajaj Iron box. I am using only my headphones to get connected in the call. I am not sure when this phone is will get busted due to this heating issue. It is definitely a challenge to hold this phone for even 1 minute. The heat that the phone is causing will definitely burn your hands and for man if you keep this phone in your pant pocket easily this will lead to infertility for you. Kindly please be aware about that.
Issue 3:
Even some unknown brands has a better touch sensitivity. The touch sensitivity is pathetic, if perform some operation it will easily take 1-2 minutes for the phone to response.
For your kind information my system has 73% of Memory free and the RAM is also 56% free.
Kindly please make this Review famous and lets make everyone aware of this issue with this phone.
Let's save people from buying this phone. There are people who don't even know what to do if this issue happens after 10 days from the date of purchase. So I feel at least this review will help people from purchasing this product in mere future."""
review_vector = vectorizer.transform([review]) # vectorizing
print(classifier_linear.predict(review_vector))

['neg']
['neg']
